In [2]:
from __future__ import print_function
import argparse
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [3]:
from Tars.distributions import Normal

In [5]:
x_dim = 20
y_dim = 30
z_dim = 40
a_dim = 50
n_batch = 2

class P1(Normal):
    def __init__(self):
        super(P1, self).__init__(cond_var=["y", "a"], var=["x"])

        self.fc1 = nn.Linear(y_dim, 10)
        self.fc2 = nn.Linear(a_dim, 10)
        self.fc21 = nn.Linear(10+10, 20)
        self.fc22 = nn.Linear(10+10, 20)

    def forward(self, a, y):
        h1 = F.relu(self.fc1(y))
        h2 = F.relu(self.fc2(a))
        h12 = torch.cat([h1, h2], 1)
        return {"loc": self.fc21(h12), "scale": F.softplus(self.fc22(h12))}

class P2(Normal):
    def __init__(self):
        super(P2, self).__init__(cond_var=["x", "y"], var=["z"])

        self.fc3 = nn.Linear(x_dim, 30)
        self.fc4 = nn.Linear(30+y_dim, 400)
        self.fc51 = nn.Linear(400, 20)
        self.fc52 = nn.Linear(400, 20)

    def forward(self, x, y):
        h3 = F.relu(self.fc3(x))
        h4 = F.relu(self.fc4(torch.cat([h3, y], 1)))
        return {"loc": self.fc51(h4), "scale": F.softplus(self.fc52(h4))}
    
p4 = Normal(loc=0, scale=1, var=["a"], dim=a_dim)
p6 = Normal(loc=0, scale=1, var=["y"], dim=y_dim)
    
x = torch.from_numpy(np.random.random((n_batch, x_dim)).astype("float32"))
y = torch.from_numpy(np.random.random((n_batch, y_dim)).astype("float32"))
a = torch.from_numpy(np.random.random((n_batch, a_dim)).astype("float32"))

In [6]:
p1 = P1()
p2 = P2()
p3 = p2 * p1
p5 = p3 * p4
p_all = p1*p2*p4*p6

In [7]:
print(p1.prob_text)
print(p2.prob_text)
print(p3.prob_text)
print(p4.prob_text)
print(p5.prob_text, p5.prob_factorized_text)
print(p_all.prob_text, p_all.prob_factorized_text)

p(x|y,a)
p(z|x,y)
p(z,x|y,a)
p(a)
p(z,x,a|y) p(z|x,y)p(x|y,a)p(a)
p(z,x,a,y) p(z|x,y)p(x|y,a)p(a)p(y)


In [8]:
for param in p3.parameters():
     print(type(param.data), param.size())

<class 'torch.Tensor'> torch.Size([10, 30])
<class 'torch.Tensor'> torch.Size([10])
<class 'torch.Tensor'> torch.Size([10, 50])
<class 'torch.Tensor'> torch.Size([10])
<class 'torch.Tensor'> torch.Size([20, 20])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([20, 20])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([30, 20])
<class 'torch.Tensor'> torch.Size([30])
<class 'torch.Tensor'> torch.Size([400, 60])
<class 'torch.Tensor'> torch.Size([400])
<class 'torch.Tensor'> torch.Size([20, 400])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([20, 400])
<class 'torch.Tensor'> torch.Size([20])


In [9]:
p1.sample({"a":a, "y":y}, return_all=False)
#p2.sample({"x":x, "y":y})
#p3.sample({"y":y, "a":a})
#p4.sample()
#p5.sample({"y":y})
#p6.sample()
#p_all.sample()

{'x': tensor([[-1.1687, -1.2349,  0.2477,  0.8112, -0.0545,  0.3289,  1.2618,
          -0.7544,  0.4870,  0.1913, -0.9061, -0.6871,  0.6354, -1.4555,
          -1.9766, -0.8279,  0.6426,  0.2607,  0.3495,  0.0896],
         [-0.3252, -0.6510,  0.0144,  2.1300,  0.6521,  0.5451, -0.9443,
           0.3575, -0.6997,  1.6965,  0.4928, -0.3187, -0.6309,  0.8209,
           0.8508,  1.0935,  0.3769, -0.7018,  0.2010,  0.2280]])}

In [10]:
outputs = p1.sample({"y":y, "a":a})
print(p1.log_likelihood(outputs))

outputs = p2.sample({"x":x, "y":y})
print(p2.log_likelihood(outputs))

outputs = p3.sample({"y":y, "a":a})
print(p3.log_likelihood(outputs))

outputs = p_all.sample(batch_size=10)
print(p_all.log_likelihood(outputs))

tensor([-26.6913, -23.7289])
tensor([-19.3082, -19.4417])
tensor([-45.8457, -40.8984])
tensor([-155.4684, -163.4326, -150.2628, -150.2103, -159.1462, -163.7559,
        -168.1021, -162.1275, -160.1595, -142.4833])
